In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/deepfake3text.csv')
df.head()

,Inputs,Label
0,The US Congress has approved proposed legislat...,0
1,"In February 2014, a man named Edward H. Lee, 2...",0
2,We will have to give up on these issues after ...,0
3,4 in. Vinyl Hardwall Box Set – 4 In. Racks Inc...,0
4,"""They put on one of these 'Millionaire's Day' ...",0


In [ ]:
from sklearn.model_selection import train_test_split
text = df['Inputs']
label = df['Label']


X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [ ]:
def tokenize_data(tokenizer, texts):

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification , AutoModel,BloomModel
from keras.layers import Embedding
tokenizer = transformers.AutoTokenizer.from_pretrained("GPT2")
model =AutoModel.from_pretrained("GPT2")
embedding_layer = Embedding(
    input_dim=model.config.vocab_size,
    output_dim=model.config.hidden_size,
    weights=[(model.get_input_embeddings().weight.detach())],
    trainable=False)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
import keras
import transformers
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x =tokenize_data(tokenizer, X_train)
x1 =tokenize_data(tokenizer, X_val)
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [ ]:
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Bidirectional
model= Sequential()
model.add(embedding_layer)
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=15,steps_per_epoch=100,batch_size=32)

Epoch 1/15
100/100 [==============================] - 6s 20ms/step - loss: 0.1696 - accuracy: 0.9497
Epoch 2/15
100/100 [==============================] - 2s 20ms/step - loss: 0.0708 - accuracy: 0.9800
Epoch 3/15
100/100 [==============================] - 2s 21ms/step - loss: 0.0612 - accuracy: 0.9812
Epoch 4/15
100/100 [==============================] - 2s 22ms/step - loss: 0.0567 - accuracy: 0.9828
Epoch 5/15
100/100 [==============================] - 2s 19ms/step - loss: 0.0518 - accuracy: 0.9837
Epoch 6/15
100/100 [==============================] - 2s 19ms/step - loss: 0.0362 - accuracy: 0.9869
Epoch 7/15
100/100 [==============================] - 2s 21ms/step - loss: 0.0360 - accuracy: 0.9903
Epoch 8/15
100/100 [==============================] - 2s 24ms/step - loss: 0.0347 - accuracy: 0.9891
Epoch 9/15
100/100 [==============================] - 2s 22ms/step - loss: 0.0279 - accuracy: 0.9912
Epoch 10/15
100/100 [==============================] - 3s 29ms/step - loss: 0.0515 - accura

In [ ]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 2s 10ms/step - loss: 0.0272 - accuracy: 0.9913


[0.02718115970492363, 0.991320013999939]

In [ ]:
text='hi'
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)
output = model.predict (prediction_input)
if output > 0.5:
  predicted_class = 'human written'
else:
  predicted_class = 'machine generated'
# Print the predicted class
print('Predicted class:',predicted_class,output)

1/1 [==============================] - 0s 62ms/step
Predicted class: human written [[0.96019024]]


In [ ]:
model.save('gpt2english.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
